## Importing Libraries and Data

In [ ]:
!pip install dropconnect-tensorflow
!pip install bert-tensorflow==1.0.1
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from dropconnect_tensorflow import DropConnectDense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub

import re
from bert import tokenization
import string
tf.gfile = tf.io.gfile

  Created wheel for dropconnect-tensorflow: filename=dropconnect_tensorflow-0.1.1-py3-none-any.whl size=4660 sha256=30cd5083856479b598fe29f165a1b17d0d1758825ad73cadb003a05b8a479cf2
  Stored in directory: /root/.cache/pip/wheels/b3/01/96/2463fe99c7de6dcdd3b28e6ecdafa4081709eb38b1a446d4dd
Successfully built dropconnect-tensorflow
     |████████████████████████████████| 67 kB 3.2 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train = pd.read_csv("/content/drive/MyDrive/python/train.csv")
test = pd.read_csv("/content/drive/MyDrive/python/test.csv")
submission = pd.read_csv("/content/drive/MyDrive/python/sample_submission.csv")

## Helper functions

In [ ]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    dense_layer1 = Dense(units=256, activation='relu')(clf_output)
    dense_layer1 = DropConnectDense(units=256, prob=0.5, activation='relu', use_bias=True)(dense_layer1)
    out = Dense(1, activation='sigmoid')(dense_layer1)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

## Data cleaning

In [ ]:
def lowercase_text(text):
    return text.lower()

train.text=train.text.apply(lambda x: lowercase_text(x))
test.text=test.text.apply(lambda x: lowercase_text(x))

In [ ]:
def remove_noise(text):
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

train.text=train.text.apply(lambda x: remove_noise(x))
test.text=test.text.apply(lambda x: remove_noise(x))

In [ ]:
train.text.head(5)

0    our deeds are the reason of this earthquake ma...
1                forest fire near la ronge sask canada
2    all residents asked to shelter in place are be...
3     people receive wildfires evacuation orders in...
4    just got sent this photo from ruby alaska as s...
Name: text, dtype: object

## Pre-Training (BERT)

In [ ]:
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [ ]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [ ]:
train_input = bert_encode(train.text.values, tokenizer, max_len=160)
test_input = bert_encode(test.text.values, tokenizer, max_len=160)
train_labels = train.target.values

In [ ]:
model = build_model(bert_layer, max_len=160)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [ ]:
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)
train_history = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=8,
    callbacks=[checkpoint],
    batch_size=10
)

Epoch 1/8
609/609 [==============================] - 1278s 2s/step - loss: 0.4402 - accuracy: 0.8105 - val_loss: 0.4228 - val_accuracy: 0.8240
Epoch 2/8
609/609 [==============================] - 1239s 2s/step - loss: 0.3137 - accuracy: 0.8800 - val_loss: 0.3888 - val_accuracy: 0.8359
Epoch 3/8
609/609 [==============================] - 1238s 2s/step - loss: 0.2010 - accuracy: 0.9286 - val_loss: 0.5230 - val_accuracy: 0.8129
Epoch 4/8
609/609 [==============================] - 1238s 2s/step - loss: 0.1214 - accuracy: 0.9550 - val_loss: 0.5460 - val_accuracy: 0.8273
Epoch 5/8
609/609 [==============================] - 1237s 2s/step - loss: 0.0840 - accuracy: 0.9703 - val_loss: 0.5775 - val_accuracy: 0.8339
Epoch 6/8
609/609 [==============================] - 1238s 2s/step - loss: 0.0627 - accuracy: 0.9755 - val_loss: 0.7744 - val_accuracy: 0.8253
Epoch 7/8
609/609 [==============================] - 1237s 2s/step - loss: 0.0530 - accuracy: 0.9767 - val_loss: 0.8236 - val_accuracy: 0.8253

In [ ]:
model.load_weights('model.h5')
test_pred = model.predict(test_input)

In [ ]:
submission['target'] = test_pred.round().astype(int)
submission.to_csv('submission.csv', index=False)